In [ ]:
"# Databricks notebook source\n# MAGIC %md\n# MAGIC # ETL Process for Guardian Data\n# MAGIC This notebook performs an ETL process on data from Unity Catalog tables, integrating and transforming data to create a comprehensive customer 360 view.\n\n# COMMAND ----------\n\nimport pyspark.sql.functions as F\nimport logging\n\n# Set up logging\nlogging.basicConfig(level=logging.INFO)\nlogger = logging.getLogger(__name__)\n\n# COMMAND ----------\n\ntry:\n    # Step 1: Data Source Configuration\n    logger.info(\"Loading data from Unity Catalog tables.\")\n    policy_df = spark.table(\"genai_demo.guardian.policy\")\n    claims_df = spark.table(\"genai_demo.guardian.claims\")\n    demographics_df = spark.table(\"genai_demo.guardian.demographics\")\n    scores_df = spark.table(\"genai_demo.guardian.scores\")\n    aiml_insights_df = spark.table(\"genai_demo.guardian.aiml_insights\")\n\n# COMMAND ----------\n\n    # Step 2: Data Selection and Filtering\n    logger.info(\"Selecting relevant fields from each dataset.\")\n    demographics_selected = demographics_df.select(\n        \"Customer_ID\", \"Customer_Name\", \"Email\", \"Phone_Number\", \"Address\", \"City\", \"State\", \"Postal_Code\",\n        \"Date_of_Birth\", \"Gender\", \"Marital_Status\", \"Occupation\", \"Income_Level\", \"Customer_Segment\"\n    )\n    claims_selected = claims_df.select(\n        \"Claim_ID\", \"Policy_ID\", \"Claim_Date\", \"Claim_Type\", \"Claim_Status\", \"Claim_Amount\", \"Claim_Payout\"\n    )\n\n# COMMAND ----------\n\n    # Step 3: Data Integration\n    logger.info(\"Joining datasets on common identifiers.\")\n    customer_policy_df = demographics_selected.join(policy_df, \"Customer_ID\")\n    # Check for duplicate columns in join\n    claims_selected = claims_selected.drop(\"Policy_ID\")  # Drop duplicate column from right DataFrame\n    customer_policy_claims_df = customer_policy_df.join(claims_selected, \"Policy_ID\")\n\n# COMMAND ----------\n\n    # Step 4: Data Aggregation\n    logger.info(\"Calculating aggregated metrics.\")\n    aggregated_df = customer_policy_claims_df.groupBy(\"Customer_ID\").agg(\n        F.count(\"Claim_ID\").alias(\"Total_Claims\"),\n        F.countDistinct(\"Policy_ID\").alias(\"Policy_Count\"),\n        F.max(\"Claim_Date\").alias(\"Recent_Claim_Date\"),\n        F.avg(\"Claim_Amount\").alias(\"Average_Claim_Amount\")\n    )\n\n# COMMAND ----------\n\n    # Step 5: Custom Calculations\n    logger.info(\"Implementing custom calculations.\")\n    final_df = aggregated_df.join(demographics_selected, \"Customer_ID\")\n    final_df = final_df.withColumn(\"Age\", F.datediff(F.current_date(), F.col(\"Date_of_Birth\")) / 365)\n    final_df = final_df.withColumn(\"Claim_To_Premium_Ratio\", F.when(F.col(\"total_premium_paid\") != 0, F.col(\"Average_Claim_Amount\") / F.col(\"total_premium_paid\")).otherwise(0))\n    final_df = final_df.withColumn(\"Claims_Per_Policy\", F.when(F.col(\"Policy_Count\") != 0, F.col(\"Total_Claims\") / F.col(\"Policy_Count\")).otherwise(0))\n    final_df = final_df.withColumn(\"Retention_Rate\", F.lit(0.85))\n    final_df = final_df.withColumn(\"Cross_Sell_Opportunities\", F.lit(\"Multi-Policy Discount, Home Coverage Add-on\"))\n    final_df = final_df.withColumn(\"Upsell_Potential\", F.lit(\"Premium Vehicle Coverage\"))\n\n# COMMAND ----------\n\n    # Step 6: Final Data Assembly\n    logger.info(\"Combining all processed data into a single dataset.\")\n    customer_360_df = final_df.join(scores_df, \"Customer_ID\").join(aiml_insights_df, \"Customer_ID\")\n\n# COMMAND ----------\n\n    # Step 7: Output Data\n    logger.info(\"Writing the final dataset to Unity Catalog table.\")\n    customer_360_df.write.format(\"delta\").mode(\"overwrite\").saveAsTable(\"genai_demo.guardian.customer_360\")\n\n    logger.info(\"ETL process completed successfully.\")\n\nexcept Exception as e:\n    logger.error(\"An error occurred during the ETL process: %s\", e)\n    logger.exception(\"Stack trace:\")\n    raise\n"
